In [17]:
!{__import__('sys').executable} -m pip install --quiet --upgrade -r requirements.txt

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-gpu 2.6.0 requires numpy~=1.19.2, but you have numpy 1.22.4 which is incompatible.


In [1]:
session_storage = 'downloads'

import os
os.makedirs(session_storage, exist_ok=True)

In [2]:
import fsspec

fs_dataset = fsspec.filesystem(
    'simplecache', 
    target_protocol='https', 
    cache_storage=session_storage,
)

In [3]:
import pandas as pd
'''
import swifter
swifter.set_defaults(
    npartitions=None,
    dask_thres=1,
    scheduler='processes',
    progress_bar=True,
    progress_bar_desc=None,
    allow_dask_on_strings=True,
    force_parallel=False
)
'''

import sklearn as skl
import sklearn.base
import sklearn.compose
#import sklearn.ensemble
import sklearn.feature_extraction
import sklearn.linear_model
import sklearn.metrics
import sklearn.model_selection
import sklearn.pipeline
import sklearn.svm

import xgboost

import numpy as np

In [4]:
import sklearnex
sklearnex.patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


## Dataset

In [10]:
fsspec.compression.compr

{None: <function fsspec.compression.noop_file(file, mode, **kwargs)>,
 'zip': <function fsspec.compression.unzip(infile, mode='rb', filename=None, **kwargs)>,
 'bz2': bz2.BZ2File,
 'gzip': <function fsspec.compression.<lambda>(f, **kwargs)>,
 'lzma': lzma.LZMAFile,
 'xz': lzma.LZMAFile}

In [142]:
import re
re.match(fr'''(?P<key>[^:]*):(?P<value>.*)''', 'PL: huge step in her relationship: \nTennesha reveals to the group that she and\n\n', flags=re.DOTALL).groupdict()

{'key': 'PL',
 'value': ' huge step in her relationship: \nTennesha reveals to the group that she and\n\n'}

In [ ]:
class IMDBList:
    def __init__(self, fo):
        self.fo = fo

    def parse(self):
        while True:
            l = self.fo.
        pass




In [156]:
import re

def _parse_imdb_list(fo, n_jobs=-1):
    pattern = re.compile(
        fr'^(?P<key>[^:]+):\s+(?P<value>.*)$', 
        flags=re.DOTALL
    )
    for l in fo.read().splitlines():
        m = re.match(pattern, l)
        if not m:
            continue
        yield m.groupdict()

In [157]:
dataset_plots = None
with fs_dataset.open(
    'https://ftp.funet.fi/pub/mirrors/ftp.imdb.com/pub/frozendata/plot.list.gz', 
    compression='gzip', 
    mode='rt', errors='ignore'
) as f:
    def _parse_imdb_list_plot(f):
        r = None

        for entry in _parse_imdb_list(f):
            # 
            if entry['key'] == 'MV':
                if r is not None:
                    yield r

                r = dict()
                r['name'] = entry['value']

            if r is None:
                # error
                continue

            if entry['key'] == 'BY':
                r.setdefault('authors', [])
                r['authors'].append(entry['value'])

            if entry['key'] == 'PL':
                r.setdefault('plot_text', [])
                r['plot_text'].append(entry['value'])

    dataset_plots = pd.DataFrame(
        list(_parse_imdb_list_plot(f))
    )

In [158]:
dataset_plots.iloc[0]['plot_text']

['#7dayslater is an interactive comedy series featuring an ensemble cast of',
 'YouTube celebrities. Each week the audience writes the brief via social',
 'media for an all-new episode featuring a well-known guest-star. Seven days',
 "later that week's episode premieres on TV and across multiple platforms."]

In [160]:
dataset_plots.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 596426 entries, 0 to 596425
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   name       596426 non-null  object
 1   plot_text  596426 non-null  object
 2   authors    506486 non-null  object
dtypes: object(3)
memory usage: 13.7+ MB
